In [1]:
#import the library sqlite3
import sqlite3


#create a new database and open a database connection with the name example
con = sqlite3.connect("database.db")

# a data cursor cur in order execute SQL statements and fetch results from SQL queries
cur = con.cursor()

# Create table with the column date, side, symbol, qty, price
try:
    cur.execute("CREATE TABLE products(date, side, symbol, qty, price)")
except sqlite3.OperationalError: 
    print("La table 'products' existe déjà")

# Insert 3 row of data: the stock APPLE ( AAPL) with the date of transaction, the nature of side ( BUY or SELL),
#the quantity of the stock traded and the price
cur.execute("""INSERT INTO products VALUES ('2021-01-01', 'BUY', 'AAPL', 100, 148.48),
                                            ('2021-01-02', 'SELL', 'AAPL', 100, 149.11),
                                            ('2021-01-03', 'BUY', 'AAPL', 75, 144.27),
                                            ('2021-01-04', 'SELL', 'AAPL', '75', '146.39')""")

# Save (commit) the changes
con.commit()

#return all resulting rows
for row in cur.execute("""SELECT * FROM products"""):
    print(row)
# Close the connection
con.close()

La table 'products' existe déjà
('2021-01-01', 'BUY', 'AAPL', 100, 148.48)
('2021-01-02', 'SELL', 'AAPL', 100, 149.11)
('2021-01-03', 'BUY', 'AAPL', 75, 144.27)
('2021-01-04', 'SELL', 'AAPL', '75', '146.39')
('2021-01-01', 'BUY', 'AAPL', 100, 148.48)
('2021-01-02', 'SELL', 'AAPL', 100, 149.11)
('2021-01-03', 'BUY', 'AAPL', 75, 144.27)
('2021-01-04', 'SELL', 'AAPL', '75', '146.39')
('2021-01-01', 'BUY', 'AAPL', 100, 148.48)
('2021-01-02', 'SELL', 'AAPL', 100, 149.11)
('2021-01-03', 'BUY', 'AAPL', 75, 144.27)
('2021-01-04', 'SELL', 'AAPL', '75', '146.39')
('2021-01-01', 'BUY', 'AAPL', 100, 148.48)
('2021-01-02', 'SELL', 'AAPL', 100, 149.11)
('2021-01-03', 'BUY', 'AAPL', 75, 144.27)
('2021-01-04', 'SELL', 'AAPL', '75', '146.39')


## the construction of a financial database

Let’s start simple and only consider the structure for the equity data that we are gathering for now – in our case the constituents of the S&P500 index. We can tag on the various tables and possibly whole databases to the mix as and when we need to, i.e. when we start to deal with the spot currency data and fund data that we plan to include.

the  plan is to segregate theequity data into 4 separate tables, using unique ids and foreign keys to create the relevant relationships and links between them.

The tables will be as follows:

<img src='C:/Users/User/Desktop/Formations_Data-LAPTOP-JP680GIQ/Formations_Data-LAPTOP-JP680GIQ/Organisme_formation/Ib_cegos/AXA/MCD.png'>

<img src='C:\Users\User\Desktop\Formations_Data-LAPTOP-JP680GIQ\Formations_Data-LAPTOP-JP680GIQ\Organisme_formation\Ib_cegos\AXA\MCD.png'>

In [2]:
#import the library os and the module Error of the library sqlite3
import os
import sqlite3
from sqlite3 import Error

#the name of the database
db_name = "financial_data.db"

# On commence par supprimer la base de données si elle existe déjà, 
# pour repartir sur des bases saines :
if os.path.isfile(db_name):
  os.remove(db_name)


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    return sqlite3.connect(db_file)



def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    cur = conn.cursor()
    cur.execute(create_table_sql)

        
def main():
    
    database = db_name

    
    # create the tables according to the MCD
    sql_create_exchange_table = """CREATE TABLE exchange (
                                id VARCHAR(255) PRIMARY KEY NOT NULL,
                                name VARCHAR(255),
                                currency VARCHAR(255),
                                code VARCHAR(255)
                                )"""
    
    sql_create_company_table = """CREATE TABLE company (
                                id VARCHAR(255) PRIMARY KEY NOT NULL,
                                name VARCHAR(255),
                                industry VARCHAR(255),
                                sector VARCHAR(255),
                                hq_location VARCHAR(255),
                                security_id VARCHAR(255),
                                FOREIGN KEY(security_id) REFERENCES security_price(security_id)
                                )"""
    
    
    sql_create_security_table = """CREATE TABLE security (
                                id VARCHAR(255) PRIMARY KEY NOT NULL,
                                ticker VARCHAR(255),
                                name VARCHAR(255),
                                company_id VARCHAR(255), 
                                exchange_id VARCHAR(255),
                                FOREIGN KEY(company_id) REFERENCES company(id),
                                FOREIGN KEY(exchange_id) REFERENCES exchange(id)
                                )"""
    
    sql_create_security_price_table =  """CREATE TABLE security_price (
                                        id VARCHAR(255) PRIMARY KEY NOT NULL,
                                        date DATE,
                                        open FLOAT,
                                        high FLOAT,
                                        low FLOAT,
                                        close FLOAT,
                                        volume FLOAT,
                                        adj_close FLOAT,
                                        security_id VARCHAR(255),
                                        FOREIGN KEY(security_id) REFERENCES security(id)
                                        )"""

    
    # create a database connection
    conn = create_connection(database)
    cur = conn.cursor()
    
    # create tables
    if conn is not None:
        # create exchange table
        cur.execute(sql_create_exchange_table)
    
        # create company table
        cur.execute(sql_create_company_table)
    
        
        # create security table
        cur.execute(sql_create_security_table)
    
        
        # create security_price table
        cur.execute(sql_create_security_price_table)
        conn.commit()
        
    else:
        print("Error! cannot create the database connection.")
main()

In [3]:
#import the library pandas , requests and io
import pandas as pd
import requests
import io

In [4]:
exchange_data = pd.read_csv("https://www.iso20022.org/sites/default/files/ISO10383_MIC/ISO10383_MIC.csv", 
                            encoding='iso-8859-1' )
exchange_data.head(30)

,MIC,OPERATING MIC,OPRT/SGMT,MARKET NAME-INSTITUTION DESCRIPTION,LEGAL ENTITY NAME,LEI,MARKET CATEGORY CODE,ACRONYM,ISO COUNTRY CODE (ISO 3166),CITY,WEBSITE,STATUS,CREATION DATE,LAST UPDATE DATE,LAST VALIDATION DATE,EXPIRY DATE,COMMENTS
0,DRSP,DRSP,OPRT,EURONEXT UK - REPORTING SERVICES,EURONEXT LONDON LIMITED,969500HMVSZ0TCV65D58,APPA,NaN,GB,LONDON,WWW.EURONEXT.COM,ACTIVE,20210927,20210927,20210927.0,NaN,APPROVED PUBLICATION ARRANGEMENT.
1,XCNQ,XCNQ,OPRT,CANADIAN SECURITIES EXCHANGE,"CNSX MARKETS, INC.",NaN,RMKT,CSE LISTED,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20090427,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...
2,PURE,XCNQ,SGMT,CANADIAN SECURITIES EXCHANGE - PURE,"CNSX MARKETS, INC.",NaN,NSPD,CSE-PURE,CA,TORONTO,WWW.THECSE.COM,ACTIVE,20061225,20210927,20210927.0,NaN,FORMERLY KNOWN AS PURE (CSE OTHER LISTED) FORM...
3,ZODM,ZODM,OPRT,ZODIA MARKETS,ZODIA MARKETS HOLDINGS LIMITED,5493006GZOHISU7JTC27,NSPD,NaN,GB,LONDON,WWW.ZODIA-MARKETS.COM,ACTIVE,20210927,20210927,20210927.0,NaN,CRYPTOASSET BROKERAGE AND EXCHANGE.
4,FNFX,BGCF,SGMT,FENICS FX ECN,"FENICS FX, LLC",549300I0NONVB9S5DZ56,NSPD,NaN,US,NEW YORK,WWW.FENICSFX.COM,ACTIVE,20210927,20210927,20210927.0,NaN,ELECTRONIC COMMUNICATIONS NETWORK.
5,NORX,NORX,OPRT,NASDAQ OSLO ASA,NASDAQ OSLO ASA,5493003IZZWOHHJY1L88,RMKT,NaN,NO,OSLO,WWW.NASDAQ.COM/SOLUTIONS/EUROPEAN-COMMODITIES,ACTIVE,20110523,20210927,20210927.0,NaN,NaN
6,POSE,POSE,OPRT,PORTFOLIO STOCK EXCHANGE,"EUROPEAN DIGITAL SECURITIES EXCHANGE, S.L.",959800UP9ANDBHTKJ408,MLTF,NaN,ES,MADRID,WWW.PORTFOLIO.EXCHANGE,ACTIVE,20210927,20210927,NaN,NaN,MULTILATERAL TRADING FACILITY.
7,PUND,PUND,OPRT,PUNDION LLC,PUNDION LLC,549300FOYST42PEQ1R51,NSPD,NaN,US,NEW YORK,WWW.PUNDION.COM,ACTIVE,20210927,20210927,20210927.0,NaN,EQUITY PRODUCTS.
8,UCBG,UCBG,OPRT,UNICREDIT BULBANK AD,UNICREDIT BULBANK AD,549300Z7V2WOFIMUEK50,SINT,NaN,BG,SOFIA,WWW.UNICREDITBULBANK.BG,ACTIVE,20210927,20210927,20210927.0,NaN,SYSTEMATIC INTERNALISER.
9,NZFX,XASX,SGMT,ASX - NEW ZEALAND FUTURES & OPTIONS,AUSTRALIAN SECURITIES EXCHANGE (ASX),549300SDXT36XYPVWI79,NSPD,ASX,AU,SYDNEY,WWW.ASX.COM.AU,ACTIVE,20070924,20210927,20210927.0,NaN,NaN


In [5]:
exchange_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2561 entries, 0 to 2560
Data columns (total 17 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   MIC                                  2561 non-null   object 
 1   OPERATING MIC                        2561 non-null   object 
 2   OPRT/SGMT                            2561 non-null   object 
 3   MARKET NAME-INSTITUTION DESCRIPTION  2561 non-null   object 
 4   LEGAL ENTITY NAME                    495 non-null    object 
 5   LEI                                  1833 non-null   object 
 6   MARKET CATEGORY CODE                 2559 non-null   object 
 7   ACRONYM                              890 non-null    object 
 8   ISO COUNTRY CODE (ISO 3166)          2560 non-null   object 
 9   CITY                                 2559 non-null   object 
 10  WEBSITE                              2387 non-null   object 
 11  STATUS                        

In [6]:
exchange_data = exchange_data[['ISO COUNTRY CODE (ISO 3166)', 'MIC', 'MARKET NAME-INSTITUTION DESCRIPTION', 'ACRONYM']]
exchange_data.head()


,ISO COUNTRY CODE (ISO 3166),MIC,MARKET NAME-INSTITUTION DESCRIPTION,ACRONYM
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [7]:
exchange_data.rename(columns={'ISO COUNTRY CODE (ISO 3166)' : 'country_code', 
                              'MIC':'code', 
                              'MARKET NAME-INSTITUTION DESCRIPTION': 'name',
                              'ACRONYM':'acronym'}, inplace=True)
exchange_data.head()

,country_code,code,name,acronym
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE
3,GB,ZODM,ZODIA MARKETS,NaN
4,US,FNFX,FENICS FX ECN,NaN


In [8]:
exchange_data['id'] = exchange_data.index
exchange_data.head()

,country_code,code,name,acronym,id
0,GB,DRSP,EURONEXT UK - REPORTING SERVICES,NaN,0
1,CA,XCNQ,CANADIAN SECURITIES EXCHANGE,CSE LISTED,1
2,CA,PURE,CANADIAN SECURITIES EXCHANGE - PURE,CSE-PURE,2
3,GB,ZODM,ZODIA MARKETS,NaN,3
4,US,FNFX,FENICS FX ECN,NaN,4


In [9]:

# exchange_data[['id', 'name', 'currency', 'code']].head(30)

def calc_new_col(row):
    match(row['country_code']):
        case "US": return "USD"
        case "GB": return "GBP"
        case "DE": return "EUR"


exchange_data["currency"] = exchange_data.apply(calc_new_col, axis=1)
print(exchange_data.head())


  country_code  code                                 name     acronym  id  \
0           GB  DRSP    EURONEXT  UK - REPORTING SERVICES         NaN   0   
1           CA  XCNQ         CANADIAN SECURITIES EXCHANGE  CSE LISTED   1   
2           CA  PURE  CANADIAN SECURITIES EXCHANGE - PURE    CSE-PURE   2   
3           GB  ZODM                        ZODIA MARKETS         NaN   3   
4           US  FNFX                        FENICS FX ECN         NaN   4   

  currency  
0      GBP  
1     None  
2     None  
3      GBP  
4      USD  


In [10]:
# save the data in a sql table exchange in the data db_name
conn = sqlite3.connect(db_name)
df_to_save = exchange_data[["id", "name", "currency", "code"]].set_index("id")
df_to_save.to_sql("exchange", conn, if_exists="replace")

2561

In [11]:
# scrape wiki table with symbols and details of Dow Jones constituents
dj_constituents = pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average#Components')[1]
dj_constituents.rename(columns={'Company':'name', 'Industry': 'industry', 'Symbol': 'ticker'}, inplace=True)
dj_constituents.head(30)

,name,Exchange,ticker,industry,Date added,Notes,Index weighting
0,3M,NYSE,MMM,Conglomerate,1976-08-09,As Minnesota Mining and Manufacturing,2.88%
1,American Express,NYSE,AXP,Financial services,1982-08-30,NaN,3.56%
2,Amgen,NASDAQ,AMGN,Biopharmaceutical,2020-08-31,NaN,4.88%
3,Apple,NASDAQ,AAPL,Information technology,2015-03-19,NaN,3.15%
4,Boeing,NYSE,BA,Aerospace and defense,1987-03-12,NaN,3.40%
5,Caterpillar,NYSE,CAT,Construction and Mining,1991-05-06,NaN,4.19%
6,Chevron,NYSE,CVX,Petroleum industry,2008-02-19,Also 1930-07-18 to 1999-11-01,3.05%
7,Cisco,NASDAQ,CSCO,Information technology,2009-06-08,NaN,1.00%
8,Coca-Cola,NYSE,KO,Drink industry,1987-03-12,Also 1932-05-26 to 1935-11-20,1.28%
9,Disney,NYSE,DIS,Broadcasting and entertainment,1991-05-06,NaN,2.32%


In [12]:
# scrape wiki table with symbols and details of s&P500 constituents
sp_constituents = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]#.to_csv('constintuents_current.csv', index=False)
sp_constituents.rename(columns={'Symbol': 'ticker', 'Security':'name', 'Headquarters Location': 'hq_location',
                                'GICS Sector': 'sector', 'GICS Sub-Industry':'industry'},inplace=True)
sp_constituents.head()

,ticker,name,SEC filings,sector,industry,hq_location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [13]:
#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'name', 'industry', 'sector', 'hq_location'

#drop duplicate
#sort
#reset index
#create the colum id based on the index

company_table = pd.concat([dj_constituents, sp_constituents])
company_table.drop_duplicates(subset="name", inplace=True)
company_table.sort_values(by="name", inplace=True)
company_table.reset_index(drop=True, inplace=True)
company_table["id"] = company_table.index
company_table = company_table[["id", "name", "industry", "sector", "hq_location"]]

In [14]:
company_table

,id,name,industry,sector,hq_location
0,0,3M,Conglomerate,NaN,NaN
1,1,A. O. Smith,Building Products,Industrials,"Milwaukee, Wisconsin"
2,2,ADM,Agricultural Products,Consumer Staples,"Chicago, Illinois"
3,3,ADP,Data Processing & Outsourced Services,Information Technology,"Roseland, New Jersey"
4,4,AES Corporation,Independent Power Producers & Energy Traders,Utilities,"Arlington, Virginia"
...,...,...,...,...,...
509,509,Zebra Technologies,Electronic Equipment & Instruments,Information Technology,"Lincolnshire, Illinois"
510,510,Zimmer Biomet,Health Care Equipment,Health Care,"Warsaw, Indiana"
511,511,Zions Bancorporation,Regional Banks,Financials,"Salt Lake City, Utah"
512,512,Zoetis,Pharmaceuticals,Health Care,"Parsippany, New Jersey"


In [15]:

#concetenate the previous tables/dataframe sp_constituents & dj_constituents with  the columns'ticker', 'name'
#drop duplicate
#sort
#reset index
#create the colum id based on the index


security_table = pd.concat([dj_constituents, sp_constituents])
security_table.drop_duplicates(subset="ticker", inplace=True)
security_table.sort_values(by="ticker", inplace=True)
security_table.reset_index(drop=True, inplace=True)
security_table["id"] = security_table.index
security_table = security_table[["id", "ticker", "name"]]

In [16]:
security_table

,id,ticker,name
0,0,A,Agilent Technologies
1,1,AAL,American Airlines Group
2,2,AAP,Advance Auto Parts
3,3,AAPL,Apple
4,4,ABBV,AbbVie
...,...,...,...
498,498,YUM,Yum! Brands
499,499,ZBH,Zimmer Biomet
500,500,ZBRA,Zebra Technologies
501,501,ZION,Zions Bancorporation


In [17]:
#create the foreign key :  populating the security table with the company_id
#first step : create a dictionary with key, value pairs being the name of the company and the corresponding id 
#within its own table, which will be our foreign key in the second table.

company_id_mapper = {k:v for k,v in zip(company_table["name"],company_table["id"])}
for company in company_id_mapper: 
    print(company, company_id_mapper[company])

3M 0
A. O. Smith 1
ADM 2
ADP 3
AES Corporation 4
AMD 5
APA Corporation 6
AT&T 7
AbbVie 8
Abbott 9
Abiomed 10
Accenture 11
Activision Blizzard 12
Adobe Inc. 13
Advance Auto Parts 14
Aflac 15
Agilent Technologies 16
Air Products and Chemicals 17
Akamai 18
Alaska Air Group 19
Albemarle Corporation 20
Alexandria Real Estate Equities 21
Align Technology 22
Allegion 23
Alliant Energy 24
Allstate 25
Alphabet Inc. (Class A) 26
Alphabet Inc. (Class C) 27
Altria 28
Amazon 29
Amcor 30
Ameren 31
American Airlines Group 32
American Electric Power 33
American Express 34
American International Group 35
American Tower 36
American Water Works 37
Ameriprise Financial 38
AmerisourceBergen 39
Ametek 40
Amgen 41
Amphenol 42
Analog Devices 43
Ansys 44
Aon 45
Apple 46
Apple Inc. 47
Applied Materials 48
Aptiv 49
Arch Capital Group 50
Arista Networks 51
Arthur J. Gallagher & Co. 52
Assurant 53
Atmos Energy 54
AutoZone 55
Autodesk 56
AvalonBay Communities 57
Avery Dennison 58
BNY Mellon 59
Baker Hughes 60
Ball 

In [18]:
#Seconde step: create the foreign key company id 

def find_company_id(company_name):
    return company_id_mapper[company_name]

security_table["company_id"] = security_table["name"].apply(find_company_id)
print(security_table)

      id ticker                     name  company_id
0      0      A     Agilent Technologies          16
1      1    AAL  American Airlines Group          32
2      2    AAP       Advance Auto Parts          14
3      3   AAPL                    Apple          46
4      4   ABBV                   AbbVie           8
..   ...    ...                      ...         ...
498  498    YUM              Yum! Brands         508
499  499    ZBH            Zimmer Biomet         510
500  500   ZBRA       Zebra Technologies         509
501  501   ZION     Zions Bancorporation         511
502  502    ZTS                   Zoetis         512

[503 rows x 4 columns]


In [19]:
#create the foreign key : populate the company table with the security_id:
ticker_id_mapper = {k:v for k,v in zip(security_table["company_id"], security_table["id"])}
    
def find_security_id(company_id):
    if company_id in ticker_id_mapper:
        return ticker_id_mapper[company_id]

company_table["security_id"] = company_table["id"].apply(find_security_id)
print(company_table)

      id                  name                                      industry  \
0      0                    3M                                  Conglomerate   
1      1           A. O. Smith                             Building Products   
2      2                   ADM                         Agricultural Products   
3      3                   ADP         Data Processing & Outsourced Services   
4      4       AES Corporation  Independent Power Producers & Energy Traders   
..   ...                   ...                                           ...   
509  509    Zebra Technologies            Electronic Equipment & Instruments   
510  510         Zimmer Biomet                         Health Care Equipment   
511  511  Zions Bancorporation                                Regional Banks   
512  512                Zoetis                               Pharmaceuticals   
513  513                  eBay            Internet & Direct Marketing Retail   

                     sector            

In [20]:
#Reda the nyse.csv and the nasdaq.csv
nyse = pd.read_csv("nyse.csv")
nyse.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,DDD,3D Systems Corporation,7.66,$927.89M,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd,NaN
1,MMM,3M Company,169.80,$97.81B,NaN,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm,NaN
2,WBAI,500.com Limited,2.95,$126.85M,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai,NaN
3,EGHT,8x8 Inc,16.75,$1.75B,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/eght,NaN
4,AHC,A.H. Belo Corporation,1.44,$34.39M,NaN,Consumer Services,Newspapers/Magazines,https://old.nasdaq.com/symbol/ahc,NaN


In [21]:
nasdaq = pd.read_csv("nasdaq.csv")
nasdaq.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9
0,VCVC,10X Capital Venture Acquisition Corp,NaN,0.000000e+00,NaN,NaN,NaN,NaN,https://old.nasdaq.com/symbol/vcvc,NaN
1,VCVCU,10X Capital Venture Acquisition Corp,NaN,0.000000e+00,NaN,NaN,NaN,NaN,https://old.nasdaq.com/symbol/vcvcu,NaN
2,TXG,"10x Genomics, Inc.",142.27,1.520301e+10,NaN,2019.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments,https://old.nasdaq.com/symbol/txg,NaN
3,YI,"111, Inc.",7.00,1.785821e+08,25511734.0,2018.0,Health Care,Medical/Nursing Services,https://old.nasdaq.com/symbol/yi,NaN
4,PIH,"1347 Property Insurance Holdings, Inc.",3.05,1.506076e+07,NaN,2014.0,Finance,Property-Casualty Insurers,https://old.nasdaq.com/symbol/pih,NaN


In [22]:
#populate the exchange_id column : assign the correct exchange_id for each stock in the security table:
# the exchange_id will be 1300 for the nyse stocks and 367 for the nasdaq stocks

nyse_companies = nyse["Name"].values
nasdaq_companies = nasdaq["Name"].values

def find_exchange_id(company_name):
    if company_name in nyse_companies: return 1300
    if company_name in nasdaq_companies: return 367


security_table["exchange_id"] = security_table["name"].apply(find_exchange_id)
security_table

,id,ticker,name,company_id,exchange_id
0,0,A,Agilent Technologies,16,NaN
1,1,AAL,American Airlines Group,32,NaN
2,2,AAP,Advance Auto Parts,14,NaN
3,3,AAPL,Apple,46,NaN
4,4,ABBV,AbbVie,8,NaN
...,...,...,...,...,...
498,498,YUM,Yum! Brands,508,NaN
499,499,ZBH,Zimmer Biomet,510,NaN
500,500,ZBRA,Zebra Technologies,509,NaN
501,501,ZION,Zions Bancorporation,511,NaN


In [23]:
#save the data in the sql table security and company of the database db_name
conn = sqlite3.connect(db_name)

security_table.set_index("id").to_sql("security", conn, if_exists="replace")
company_table.set_index("id").to_sql("company", conn, if_exists="replace")


514

In [24]:
#Extraction the prices of YAHOO: WARNING this action may take 15 min !!!

import time
import pandas_datareader.data as web
from ipywidgets import IntProgress
from IPython.display import display

stock_pricing_dfs = []
max_count = len(security_table["id"])
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

for stock_id in security_table['id']:
    try:
        stock_pricing_df = web.DataReader(security_table.iloc[stock_id]['ticker'], 
                           start='2010-1-1', 
                           end='2020-08-31', 
                           data_source='yahoo')
        stock_pricing_df['security_id'] = stock_id
        stock_pricing_dfs.append(stock_pricing_df)
    except:
        pass
    
    f.value += 1 # signal to increment the progress bar

security_price_table = pd.concat(stock_pricing_dfs)


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


In [27]:
security_price_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1276281 entries, 0 to 1276280
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Date         1276281 non-null  datetime64[ns]
 1   high         1276281 non-null  float64       
 2   low          1276281 non-null  float64       
 3   open         1276281 non-null  float64       
 4   close        1276281 non-null  float64       
 5   volume       1276281 non-null  float64       
 6   adj_close    1276281 non-null  float64       
 7   security_id  1276281 non-null  int64         
 8   id           1276281 non-null  int64         
dtypes: datetime64[ns](1), float64(6), int64(2)
memory usage: 87.6 MB


In [26]:
#insert data in the table security_price
security_price_table.columns = ['high', 'low', 'open', 'close', 'volume', 'adj_close', 'security_id']
security_price_table.reset_index(inplace=True)
security_price_table['id'] = security_price_table.index
print(security_price_table.head())
security_price_table.to_sql("security_price", conn, if_exists="append", index=False)

        Date       high        low       open      close     volume  \
0 2010-01-04  22.625179  22.267525  22.453505  22.389128  3815561.0   
1 2010-01-05  22.331903  22.002861  22.324751  22.145924  4186031.0   
2 2010-01-06  22.174536  22.002861  22.067240  22.067240  3243779.0   
3 2010-01-07  22.045780  21.816881  22.017166  22.038628  3095172.0   
4 2010-01-08  22.067240  21.745352  21.917025  22.031473  3733918.0   

   adj_close  security_id  id  
0  20.434929            0   0  
1  20.212959            0   1  
2  20.141132            0   2  
3  20.115025            0   3  
4  20.108498            0   4  


1276281